# Geotagged Only

In [2]:
################ GEOTAGGED ONLY ##########################

# get distances column from all datasets
# first determine if there are any duplicate tweet ids in there
"""
import pandas as pd
import tqdm

for i in tqdm.tqdm(range(0, 112)):
    df = pd.read_feather("s3://mt5599/final/processed_tweets_" + str(i) + ".feather")

    # keep tweets with no distances and NER mention
    df = df[pd.notnull(df.place_full_name)].reset_index()
    df.to_feather("s3://mt5599/dissertation/geotagged_" + str(i) + ".feather" )
"""

'\nimport pandas as pd\nimport tqdm\n\nfor i in tqdm.tqdm(range(0, 112)):\n    df = pd.read_feather("s3://mt5599/final/processed_tweets_" + str(i) + ".feather")\n\n    # keep tweets with no distances and NER mention\n    df = df[pd.notnull(df.place_full_name)].reset_index()\n    df.to_feather("s3://mt5599/dissertation/geotagged_" + str(i) + ".feather" )\n'

In [3]:
# combining into one dataset
"""
import tqdm
import pandas as pd

dfs = []

for i in tqdm.tqdm(range(0, 112)):
    df = pd.read_feather("s3://mt5599/dissertation/geotagged_" + str(i) + ".feather")

    dfs.append(df)

df = pd.concat(dfs, axis=0, ignore_index=True).reset_index(drop=True).reset_index()
df.to_feather("s3://mt5599/final/geotagged.feather")
"""

'\nimport tqdm\nimport pandas as pd\n\ndfs = []\n\nfor i in tqdm.tqdm(range(0, 112)):\n    df = pd.read_feather("s3://mt5599/dissertation/geotagged_" + str(i) + ".feather")\n\n    dfs.append(df)\n\ndf = pd.concat(dfs, axis=0, ignore_index=True).reset_index(drop=True).reset_index()\ndf.to_feather("s3://mt5599/final/geotagged.feather")\n'

In [4]:
#df.drop(["level_0", "index"], axis=1, inplace=True)

In [ ]:
import pandas as pd

df = pd.read_feather("s3://mt5599/final/geotagged.feather")
print("read")
df = df.drop_duplicates("id").drop(["level_0", "index"], axis=1)
print("cleaning")
df

In [ ]:
#df = df0[0:100000]

In [ ]:
!pip install epiweeks
!pip install -U kaleido

In [ ]:
import tqdm
from multiprocessing import Pool # multithreading
from epiweeks import Week

# granularity
def granularity_helper(row):
    
    s = row[1]["gmaps_address"]
    
    week = Week.fromdate(row[1]["DateTime"], system="iso")
    row[1]["epi_week"] = week.cdcformat()
    return row[1]



def granularity(df):
    
    pool = Pool(processes=round(len(df.index)/10000))

    result_arr = []
    
    for result in tqdm.tqdm(pool.imap_unordered(granularity_helper, df.iterrows()),
                            total=len(df.index)):
        result_arr.append(result)
                
    df = pd.concat(result_arr, axis=1).transpose().sort_index()
                
    return df

In [ ]:
# add epi weeks into df

df = granularity(df)
df.reset_index(inplace=True)

In [ ]:
df.to_feather("s3://mt5599/final/geotagged.feather")

In [ ]:
# Investigating top 20 most geotagged countries and their frequencies

loc_vc = df.place_country.value_counts().rename_axis('unique_values').reset_index(name='counts')
loc_total = sum(loc_vc.counts)
#df_only = df[df.ner_type != "LOC"]
#only_loc_vc = df_only.place_country.value_counts().rename_axis('unique_values').reset_index(name='counts')
#only_loc_total = sum(loc_only_vc.counts)

vc = pd.DataFrame(data = {"loc": [],
                          "loc_counts": [],
                          "loc_prop": []})
                    #     "only_loc": [],
                   #      "only_loc_counts": [],
                   #      "only_loc_prop": []})

n = 20
for i in range(n):
    vc.loc[i, "loc"] = loc_vc.loc[i, "unique_values"]
    vc.loc[i, "loc_counts"] = loc_vc.loc[i, "counts"]
    vc.loc[i, "loc_prop"] = loc_vc.loc[i, "counts"] / loc_total
    
  #  vc.loc[i, "only_loc"] = only_loc_vc.loc[i, "unique_values"]
  #  vc.loc[i, "only_loc_counts"] = only_loc_vc.loc[i, "counts"]
  #  vc.loc[i, "only_loc_prop"] = only_loc_vc.loc[i, "counts"] / only_loc_total
    
vc.loc[n+1, "loc"] = "Other"
vc.loc[n+1, "loc_counts"] = loc_total - sum(loc_vc.loc[0:n, "counts"])
vc.loc[n+1, "loc_prop"] = vc.loc[n+1, "loc_counts"] / loc_total
#vc.loc[n+1, "only_loc"] = "Other"
#vc.loc[n+1, "only_loc_counts"] = only_loc_total - sum(only_loc_vc.loc[0:n, "counts"])
#vc.loc[n+1, "only_loc_prop"] = vc.loc[n+1, "only_loc_counts"] / only_loc_total

vc.reset_index(inplace=True, drop=True)
#vc.drop(0, axis=0, inplace=True)
print("Table \ref{tab:geotagged-locations} shows the number of top 20 most geotagged countries,",
      "with 'Other' denoting other countries (does not include un-geotagged tweets).")
vc

In [ ]:
print("For each user, the countries they geotagged over the course of the relevant time period were found.")

import itertools
import numpy as np

user_df = df.groupby(['username'])["place_country"].unique().reset_index(name='countries')

# number of countries per user
countries_per_user = user_df.countries.str.len()

print()
print("The mean number of geotagged countries was ", np.mean(countries_per_user),
     "and the median was ", np.median(countries_per_user))

user_countries = user_df.countries.tolist()
all_countries = pd.DataFrame({"country":list(itertools.chain.from_iterable(user_countries))})
temp = pd.DataFrame(all_countries.country.value_counts())

print("Table \ref{tab:geotagged-locations-users} shows their breakdown.")
temp

In [ ]:
# ARGENTINA - tweets

ar = df[df.place_country_code == "AR"]

loc_vc = ar.place_name.value_counts().rename_axis('unique_values').reset_index(name='counts')
loc_total = sum(loc_vc.counts)
#df_only = df[df.ner_type != "LOC"]
#only_loc_vc = df_only.place_country.value_counts().rename_axis('unique_values').reset_index(name='counts')
#only_loc_total = sum(loc_only_vc.counts)

vc = pd.DataFrame(data = {"loc": [],
                          "loc_counts": [],
                          "loc_prop": []})
                    #     "only_loc": [],
                   #      "only_loc_counts": [],
                   #      "only_loc_prop": []})

n = 20
for i in range(n):
    vc.loc[i, "loc"] = loc_vc.loc[i, "unique_values"]
    vc.loc[i, "loc_counts"] = loc_vc.loc[i, "counts"]
    vc.loc[i, "loc_prop"] = loc_vc.loc[i, "counts"] / loc_total
    
  #  vc.loc[i, "only_loc"] = only_loc_vc.loc[i, "unique_values"]
  #  vc.loc[i, "only_loc_counts"] = only_loc_vc.loc[i, "counts"]
  #  vc.loc[i, "only_loc_prop"] = only_loc_vc.loc[i, "counts"] / only_loc_total
    
vc.loc[n+1, "loc"] = "Other"
vc.loc[n+1, "loc_counts"] = loc_total - sum(loc_vc.loc[0:n, "counts"])
vc.loc[n+1, "loc_prop"] = vc.loc[n+1, "loc_counts"] / loc_total
#vc.loc[n+1, "only_loc"] = "Other"
#vc.loc[n+1, "only_loc_counts"] = only_loc_total - sum(only_loc_vc.loc[0:n, "counts"])
#vc.loc[n+1, "only_loc_prop"] = vc.loc[n+1, "only_loc_counts"] / only_loc_total

vc.reset_index(inplace=True, drop=True)
#vc.drop(0, axis=0, inplace=True)
print("Table \ref{tab:geotagged-locations} shows the number of top 20 most geotagged locations within Argentina,",
      "with 'Other' denoting other countries (does not include un-geotagged tweets).")
vc

In [ ]:
# UNITED STATES - tweets

us = df[df.place_country_code == "US"]

loc_vc = us.place_name.value_counts().rename_axis('unique_values').reset_index(name='counts')
loc_total = sum(loc_vc.counts)
#df_only = df[df.ner_type != "LOC"]
#only_loc_vc = df_only.place_country.value_counts().rename_axis('unique_values').reset_index(name='counts')
#only_loc_total = sum(loc_only_vc.counts)

vc = pd.DataFrame(data = {"loc": [],
                          "loc_counts": [],
                          "loc_prop": []})
                    #     "only_loc": [],
                   #      "only_loc_counts": [],
                   #      "only_loc_prop": []})

n = 20
for i in range(n):
    vc.loc[i, "loc"] = loc_vc.loc[i, "unique_values"]
    vc.loc[i, "loc_counts"] = loc_vc.loc[i, "counts"]
    vc.loc[i, "loc_prop"] = loc_vc.loc[i, "counts"] / loc_total
    
  #  vc.loc[i, "only_loc"] = only_loc_vc.loc[i, "unique_values"]
  #  vc.loc[i, "only_loc_counts"] = only_loc_vc.loc[i, "counts"]
  #  vc.loc[i, "only_loc_prop"] = only_loc_vc.loc[i, "counts"] / only_loc_total
    
vc.loc[n+1, "loc"] = "Other"
vc.loc[n+1, "loc_counts"] = loc_total - sum(loc_vc.loc[0:n, "counts"])
vc.loc[n+1, "loc_prop"] = vc.loc[n+1, "loc_counts"] / loc_total
#vc.loc[n+1, "only_loc"] = "Other"
#vc.loc[n+1, "only_loc_counts"] = only_loc_total - sum(only_loc_vc.loc[0:n, "counts"])
#vc.loc[n+1, "only_loc_prop"] = vc.loc[n+1, "only_loc_counts"] / only_loc_total

vc.reset_index(inplace=True, drop=True)
#vc.drop(0, axis=0, inplace=True)
print("Table \ref{tab:geotagged-locations} shows the number of top 20 most geotagged locations within Argentina,",
      "with 'Other' denoting other countries (does not include un-geotagged tweets).")
vc

In [ ]:
# BRAZIL - tweets

br = df[df.place_country_code == "BR"]

loc_vc = br.place_name.value_counts().rename_axis('unique_values').reset_index(name='counts')
loc_total = sum(loc_vc.counts)
#df_only = df[df.ner_type != "LOC"]
#only_loc_vc = df_only.place_country.value_counts().rename_axis('unique_values').reset_index(name='counts')
#only_loc_total = sum(loc_only_vc.counts)

vc = pd.DataFrame(data = {"loc": [],
                          "loc_counts": [],
                          "loc_prop": []})
                    #     "only_loc": [],
                   #      "only_loc_counts": [],
                   #      "only_loc_prop": []})

n = 20
for i in range(n):
    vc.loc[i, "loc"] = loc_vc.loc[i, "unique_values"]
    vc.loc[i, "loc_counts"] = loc_vc.loc[i, "counts"]
    vc.loc[i, "loc_prop"] = loc_vc.loc[i, "counts"] / loc_total
    
  #  vc.loc[i, "only_loc"] = only_loc_vc.loc[i, "unique_values"]
  #  vc.loc[i, "only_loc_counts"] = only_loc_vc.loc[i, "counts"]
  #  vc.loc[i, "only_loc_prop"] = only_loc_vc.loc[i, "counts"] / only_loc_total
    
vc.loc[n+1, "loc"] = "Other"
vc.loc[n+1, "loc_counts"] = loc_total - sum(loc_vc.loc[0:n, "counts"])
vc.loc[n+1, "loc_prop"] = vc.loc[n+1, "loc_counts"] / loc_total
#vc.loc[n+1, "only_loc"] = "Other"
#vc.loc[n+1, "only_loc_counts"] = only_loc_total - sum(only_loc_vc.loc[0:n, "counts"])
#vc.loc[n+1, "only_loc_prop"] = vc.loc[n+1, "only_loc_counts"] / only_loc_total

vc.reset_index(inplace=True, drop=True)
#vc.drop(0, axis=0, inplace=True)
print("Table \ref{tab:geotagged-locations} shows the number of top 20 most geotagged locations within Argentina,",
      "with 'Other' denoting other countries (does not include un-geotagged tweets).")
vc

In [ ]:
# ARGENTINA GEOTAGS - by user

ar = df[df.place_country_code == "AR"]

import itertools

user_ar = ar.groupby(['username'])["place_name"].unique().reset_index(name='countries')

# number of countries per user
countries_per_user = user_ar.countries.str.len()

#print()
#print("The mean number of geotagged placenames in Argentina was ", np.mean(countries_per_user),
#     "and the median was ", np.median(countries_per_user))

user_countries = user_ar.countries.tolist()
all_countries = pd.DataFrame({"country":list(itertools.chain.from_iterable(user_countries))})
temp = pd.DataFrame(all_countries.country.value_counts())

print("Table \ref{tab:geotagged-locations-users} shows the breakdown of top 20 most common geotags",
      "within Argentina aggregated by user to avoid the problem of varying tweeting frequencies.")
temp

In [ ]:
print("There were a total of ", ar.shape[0]," geotagged tweets from Argentina.")

In [ ]:
# UNITED STATES GEOTAGS - by user

us = df[df.place_country_code == "US"]

import itertools

user_ar = us.groupby(['username'])["place_name"].unique().reset_index(name='countries')

# number of countries per user
countries_per_user = user_ar.countries.str.len()

#print()
#print("The mean number of geotagged placenames in Argentina was ", np.mean(countries_per_user),
#     "and the median was ", np.median(countries_per_user))

user_countries = user_ar.countries.tolist()
all_countries = pd.DataFrame({"country":list(itertools.chain.from_iterable(user_countries))})
temp = pd.DataFrame(all_countries.country.value_counts())

print("Table \ref{tab:geotagged-locations-users} shows the breakdown of top 20 most common geotags",
      "within Argentina aggregated by user to avoid the problem of varying tweeting frequencies.")
temp

In [ ]:
# BRAZIL GEOTAGS - by user

br = df[df.place_country_code == "BR"]

import itertools

user_ar = br.groupby(['username'])["place_name"].unique().reset_index(name='countries')

# number of countries per user
countries_per_user = user_ar.countries.str.len()

#print()
#print("The mean number of geotagged placenames in Argentina was ", np.mean(countries_per_user),
#     "and the median was ", np.median(countries_per_user))

user_countries = user_ar.countries.tolist()
all_countries = pd.DataFrame({"country":list(itertools.chain.from_iterable(user_countries))})
temp = pd.DataFrame(all_countries.country.value_counts())

print("Table \ref{tab:geotagged-locations-users} shows the breakdown of top 20 most common geotags",
      "within Argentina aggregated by user to avoid the problem of varying tweeting frequencies.")
temp

## Epi Weeks

In [ ]:
# Countries over Epi Weeks - tweets

from itertools import product
import matplotlib.pyplot as plt

top_20 = df.place_country.value_counts().rename_axis('unique_values').reset_index(name='counts')
#top_20 = top_20.loc[,0:20]
top_20 = top_20.loc[0:20, "unique_values"].tolist()
top_20

import plotly.express as px

loc_p = df[df.place_country.isin(top_20)]
loc_p = loc_p.groupby(['place_country', "epi_week"])["id"].count().reset_index(name='counts')
loc_p = loc_p.sort_values(["epi_week"], ascending=True).reset_index(drop=True)

loc_temp = pd.DataFrame(list(product(df.place_country.unique(), df.epi_week.unique())), columns=['place_country', 'epi_week'])
loc_temp["counts"] = 0

loc_p = pd.concat([loc_p, loc_temp]).drop_duplicates(["place_country", "epi_week"])#, ignore_index=True)
loc_p = loc_p.sort_values(["epi_week"], ascending=True)

#only_loc_p = df_loc[df_loc.place_country.isin(top_20)]
#only_loc_p = only_loc_p.groupby(['place_country', "epi_week"])["id"].count().reset_index(name='counts')
# plot pie chart

fig = px.line(loc_p, x="epi_week", y="counts", color="place_country",
                 labels={
                     "epi_week": "Epi Week",
                     "counts": "Number of Tweets",
                     "place_country": "Country"})

fig.write_image("geotagged_users_country_epi_week.png")
fig.show()

In [ ]:
# Countries over Epi Weeks - users

import itertools

cepi = df.drop_duplicates(["username", "place_country", "epi_week"])
cepi = cepi.groupby(['place_country', "epi_week"])["username"].count().reset_index(name='counts')
cepi = cepi.sort_values(["epi_week"], ascending=True).reset_index(drop=True)


top_20 = cepi.place_country.value_counts().rename_axis('unique_values').reset_index(name='counts')
top_20 = top_20.loc[0:20, "unique_values"].tolist()

import plotly.express as px

loc_p = cepi[cepi.place_country.isin(top_20)]
#loc_p = loc_p.groupby(['place_country', "epi_week"])["id"].count().reset_index(name='counts')
#loc_p = loc_p.sort_values(["epi_week"], ascending=True).reset_index(drop=True)

loc_temp = pd.DataFrame(list(product(cepi.place_country.unique(), df.epi_week.unique())), columns=['place_country', 'epi_week'])
loc_temp["counts"] = 0

loc_p = pd.concat([loc_p, loc_temp]).drop_duplicates(["place_country", "epi_week"])#, ignore_index=True)
loc_p = loc_p.sort_values(["epi_week"], ascending=True)


fig = px.line(loc_p, x="epi_week", y="counts", color="place_country",
                 labels={
                     "epi_week": "Epi Week",
                     "counts": "Number of Users in Country",
                     "place_country": "Country"})

fig.write_image("geotagged_users_country_epi_week.png")
fig.show()

In [ ]:
# lower value counts of Place Object?